In [87]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
load_dotenv()

True

In [88]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="the sentiment of the text")

In [89]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=0)        

In [90]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [91]:
structured_model = model.with_structured_output(SentimentSchema)
structured_model2 = model.with_structured_output(DiagnosisSchema)

In [92]:
class Reveiwinputstate(TypedDict):
    review: str 
    sentiment: Literal["positive", "negative"]
    diagnosis: dict
    response: str
   
    

In [93]:
def sentiment_analysis(state:Reveiwinputstate):
    prompt = f"Determine the sentiment of the following review: {state["review"]}. Respond with either 'positive' or 'negative'."
    response = structured_model.invoke(prompt).sentiment
    return {"sentiment": response}

In [94]:
def positive_response(state: Reveiwinputstate):

    prompt = f"""Write a warm thank-you message in response to this review:\n\n\"{state['review']}\"\nAlso, kindly ask the user to leave feedback on our website."""
    
    response = model.invoke(prompt).content

    return {'response': response}

In [95]:
def run_diagnosis(state: Reveiwinputstate):
    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n""Return issue_type, tone, and urgency."""
    response = structured_model2.invoke(prompt)
    return {'diagnosis': response.model_dump()}

In [96]:
def negative_response(state: Reveiwinputstate):

    diagnosis = state['diagnosis']
    prompt = f"""You are a support assistant.The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.Write an empathetic, helpful resolution message."""
    response = model.invoke(prompt).content

    return {'response': response}

In [97]:
def check_sentiment(state: Reveiwinputstate) -> Literal["positive_response", "run_diagnosis"]:

    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'

In [98]:
graph = StateGraph(Reveiwinputstate)

In [99]:
graph.add_node("sentiment_analysis",sentiment_analysis)
graph.add_node("positive_response",positive_response)
graph.add_node("run_diagnosis",run_diagnosis)   
graph.add_node("negative_response",negative_response)
graph.add_node("check_sentiment",check_sentiment)
graph.add_edge(START, 'sentiment_analysis')
graph.add_conditional_edges('sentiment_analysis', check_sentiment)
graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()


In [100]:
initial_state={
    'review': "I've been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality."
}
workflow.invoke(initial_state)

{'review': "I've been trying to log in for over an hour now, and the app keeps freezing on the authentication screen. I even tried reinstalling it, but no luck. This kind of bug is unacceptable, especially when it affects basic functionality.",
 'sentiment': 'negative',
 'diagnosis': {'issue_type': 'Bug', 'tone': 'frustrated', 'urgency': 'high'},
 'response': "Okay, I understand you're experiencing a bug and are understandably frustrated. I see that you've marked this as high urgency, and I want to assure you that I'm taking this seriously and will do everything I can to help resolve this for you as quickly as possible.\n\nFirst, I want to apologize for the inconvenience this is causing. Bugs can be incredibly disruptive, and I know how frustrating it is when something isn't working as expected.\n\nTo help me understand the issue better and find the best solution, could you please provide me with a little more information? Specifically:\n\n*   **What exactly is the bug you're experienc